In [12]:
import torch #import necessary for data processing
import numpy as np 
import torch.nn as nn
import matplotlib.pyplot as plt
import csv
import nerfvis
import os
import pickle #used to get data in the proper format 
import pillow_heif
import cv2.aruco as aruco
import cv2
from PIL import Image
from nerfvis import scene #Used to visualize nerf, delete if NerfStudio is used instead 
from tqdm import tqdm
from torch.utils.data import DataLoader
from pillow_heif import register_heif_opener

In [ ]:
# Re-load the image using PIL
image_path = '/mnt/data/Test.jpg'
image = Image.open(image_path)
image = image.convert('RGB')  # Ensure image is in RGB format

# Normalize pixel coordinates to range [-1, 1]
width, height = image.size
x = np.linspace(-1, 1, width)
y = np.linspace(1, -1, height)
xv, yv = np.meshgrid(x, y)

# Assume camera parameters
focal_length = 1.0  # for a 90 degree field of view
camera_origin = np.array([0, 0, 0])  # Camera origin

# Compute ray directions assuming the camera is looking along the z-axis
ray_directions = np.stack((xv, yv, np.full_like(xv, focal_length)), axis=-1)
ray_directions_normalized = ray_directions / np.linalg.norm(ray_directions, axis=-1, keepdims=True)

# Extract color values and normalize to range [0, 1]
pixel_colors = np.asarray(image) / 255.0

# Flatten the ray directions and colors
flattened_ray_directions = ray_directions_normalized.reshape(-1, 3)
flattened_pixel_colors = pixel_colors.reshape(-1, 3)

# Create the pixel coordinates for ray origin, assuming all rays originate from the same point (camera position)
ray_origins = np.tile(camera_origin, (flattened_ray_directions.shape[0], 1))

# Concatenate the ray origins, directions, and colors to form the final dataset
final_dataset = np.concatenate((ray_origins, flattened_ray_directions, flattened_pixel_colors), axis=1)

# Show the shape of the final dataset and the first row as a sample
final_dataset.shape, final_dataset[0]

In [9]:
#Uses Pillow
def PILimage_quality_regularizer(input_image_path, output_image_path, target_width=1000):
    with Image.open(input_image_path) as img:
        # Calculate the target height to maintain the aspect ratio
        aspect_ratio = img.height / img.width
        target_height = int(target_width * aspect_ratio)

        # Resize the image while maintaining the aspect ratio using LANCZOS filter for high quality
        img = img.resize((target_width, target_height), Image.Resampling.LANCZOS)

        # Save the resized image
        img.save(output_image_path, quality=95)  # Use high quality for JPG

# Example usage:
#image_quality_regularizer('path/to/input/image.jpg', 'path/to/output/resized_image.jpg')
PILimage_quality_regularizer("pkltestimg/JPGTest.jpg", "pkltestimg/TestReziePIL.jpg")


In [10]:
#Uses OPENCV
def CVimage_quality_regularizer(input_image_path, output_image_path, target_width=1000):
    img = cv2.imread(input_image_path)
    aspect_ratio = img.shape[0] / img.shape[1]  # height/width
    target_height = int(target_width * aspect_ratio)

    # Resize the image while maintaining the aspect ratio
    res = cv2.resize(img, (target_width, target_height), interpolation=cv2.INTER_LANCZOS4)
    # Save the resized image using high quality for JPG
    cv2.imwrite(output_image_path, res, [int(cv2.IMWRITE_JPEG_QUALITY), 95])

# Example usage:
CVimage_quality_regularizer("pkltestimg/JPGTest.jpg", "pkltestimg/TestRezieCV.jpg")

In [11]:
def resize_image_to_memory_size(input_image_path, target_memory_size_bytes, output_image_path):
    """
    Resizes an image to approximate a target memory size.

    Args:
    - input_image_path (str): The path to the input image.
    - target_memory_size_bytes (int): The target memory size in bytes.
    - output_image_path (str): The path to save the resized image.
    """
    with Image.open(input_image_path) as img:
        current_size_bytes = os.path.getsize(input_image_path)
        size_ratio = target_memory_size_bytes / current_size_bytes
        # (since the number of pixels = width * height)
        new_width = int(img.width * (size_ratio ** 0.5))
        new_height = int(img.height * (size_ratio ** 0.5))
        resized_img = img.resize((new_width, new_height), Image.LANCZOS)
        resized_img.save(output_image_path, quality=85)  # Adjust quality as needed
        # Optionally, check the size and adjust if necessary
        resized_size_bytes = os.path.getsize(output_image_path)
        print(f"Target Size: {target_memory_size_bytes} bytes, Resized Size: {resized_size_bytes} bytes")

# Example usage
resize_image_to_memory_size('pkltestimg/JPGTest.jpg', 100000, 'pkltestimg/TestRezieDATABIG.jpg')
resize_image_to_memory_size('pkltestimg/JPGTest.jpg', 25000, 'pkltestimg/TestRezieDATASMALL.jpg')

Target Size: 100000 bytes, Resized Size: 61703 bytes
Target Size: 25000 bytes, Resized Size: 24587 bytes


In [13]:
# Initialize video capture
cap = cv2.VideoCapture(0)

# Load the predefined dictionary
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_6X6_250)

# Initialize the detector parameters using default values
parameters = aruco.DetectorParameters_create()

# Calibration parameters for the camera (Assuming you have calibrated your camera)
# Replace these with your actual calibration parameters
camera_matrix = np.array([[1000, 0, 320], [0, 1000, 240], [0, 0, 1]], dtype=np.float32)
dist_coeffs = np.zeros((4, 1))  # Assuming no lens distortion

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect the markers in the image
    corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

    if ids is not None:
        # Estimate pose of each marker
        rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(corners, 0.05, camera_matrix, dist_coeffs)
        for i in range(ids.size):
            # Visualize the detected markers
            aruco.drawDetectedMarkers(frame, corners, ids)
            # Draw the pose estimation
            aruco.drawAxis(frame, camera_matrix, dist_coeffs, rvecs[i], tvecs[i], 0.03)

    # Display the resulting frame
    cv2.imshow('Frame', frame)

    # Exit loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

AttributeError: module 'cv2.aruco' has no attribute 'DetectorParameters_create'

In [ ]:
#This is a code graveyard. Do not run this. This exists for documentation ONLY. These are my failures/methods used to test failures.
'''

def convert_heic_to_jpeg(inputdir, outputdir):
    # Get list of HEIF and HEIC files in directory
    files = [f for f in os.listdir(inputdir) if f.endswith(('.heic', 'heif', '.HEIC', '.HEIF'))]
    if not os.path.exists(outputdir): 
        os.makedirs(outputdir)
    for filename in files: 
        image = Image.open(os.path.join(inputdir, filename))
        output_file_path = os.path.join(outputdir, os.path.splitext(filename)[0] + '.jpg')
        image.convert('RGB').save(output_file_path, format = 'JPEG')

def quality_regularizer(inputdir, outputdir, target_width = 1000):
    files = [f for f in os.listdir(inputdir) if f.lower().endswith(('.jpeg', '.jpg', 'png'))]
    if not os.path.exists(outputdir):
        os.makedirs(outputdir)
    for filename in files: 
        img = cv2.imread(filename)
        with Image.open(filename) as image: 
            aspect_ratio = image.height / img.width
            target_height = int(target_width * aspect_ratio)
        

def test(target_width=1000): 
    img = cv2.imread("IMGRegularizationTest/JPGTest.jpg")
    scale_factor = img.shape[0]/img.shape[1]
    res = cv2.resize(img, dsize=(576, 324),  interpolation=cv2.INTER_CUBIC)
    #res = cv2.resize(img, dsize=(target_width, int(img.shape[1]*scale_factor)), interpolation=cv2.INTER_CUBIC)
    plt.imshow(res)
    plt.show()
    print("The Height is ", img.shape[0])
    print("The Width of the image is ", img.shape[1])
    print(img.shape[2]) 

test()

def img_quality_regularizer(input_dir, output_dir, target_width = 1000): 
    files = [f for f in os.listdir(input_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for filename in files: 
        image = Image.open(os.path.join(input_dir, filename))
        output_file_path = os.path.join(output_dir, os.path.splittext(filename)[0] + '.jpg')


        ''' 
        #with Image.open(filename) as img: 
            #aspect_ratio = img.height / img.width
           # target_height = int(target_width * aspect_ratio)
            #img = img.resize((target_width, target_height), Image.Resampling.LANCZOS)
           # output_file_path = os.path.join(output_dir, os.path.splittext(filename)[0] + '_resized.jpg')
            #image.convert("RGB").save(output_file_path, format = 'JPEG')
'''
img_quality_regularizer('IMGRegularizationTest', "IMGRegularizationTest")

def image_quality_regularizer(input_image_path, output_image_path, target_width=1000):
    """
    Takes an input image (PNG or JPG) and resizes it to a new image with a width of 1000 pixels
    while maintaining the aspect ratio.

    Parameters:
    - input_image_path: str, path to the input image file.
    - output_image_path: str, path where the output image will be saved.
    - target_width: int, the target width of the output image in pixels, default is 1000.

    Returns: None
    """
    # Open the input image
    with Image.open(input_image_path) as img:
        # Calculate the target height to maintain the aspect ratio
        aspect_ratio = img.height / img.width
        target_height = int(target_width * aspect_ratio)

        # Resize the image while maintaining the aspect ratio
        img = img.resize((target_width, target_height), Image.ANTIALIAS)

        # Save the resized image
        img.save(output_image_path, quality=95)  # Use high quality for JPG

# Path for the example image
input_image_path = '/mnt/data/Test.jpg'
output_image_path = '/mnt/data/Resized_Image.jpg'

# Call the function to resize the example image
image_quality_regularizer(input_image_path, output_image_path)

'''